# Simple but Powerful: Quantum Protocols That Show Off the Weirdness

## Lecture Overview

**Key Message**: *"Quantum mechanics isn't just weird — it's useful."*

We'll explore four fundamental quantum protocols that demonstrate how entanglement, interference, and measurement work together to solve problems and move information in ways impossible classically.

### What We'll Cover:
1. **Quantum Teleportation** - Transfer unknown quantum states using entanglement
2. **Gate Teleportation** - Implement quantum gates through measurement
3. **Superdense Coding** - Send 2 classical bits using 1 qubit
4. **Deutsch-Jozsa Algorithm** - Exponential speedup in oracle problems

Let's dive in!

In [1]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Circle, Rectangle, FancyBboxPatch
import matplotlib.patches as mpatches
from IPython.display import display, HTML
import ipywidgets as widgets

# Set up plotting style
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12

# Define quantum states and gates
# Basis states
ket0 = np.array([1, 0], dtype=complex)
ket1 = np.array([0, 1], dtype=complex)

# Pauli matrices
I = np.array([[1, 0], [0, 1]], dtype=complex)
X = np.array([[0, 1], [1, 0]], dtype=complex)
Y = np.array([[0, -1j], [1j, 0]], dtype=complex)
Z = np.array([[1, 0], [0, -1]], dtype=complex)

# Hadamard gate
H = np.array([[1, 1], [1, -1]], dtype=complex) / np.sqrt(2)

# CNOT gate
CNOT = np.array([[1, 0, 0, 0],
                 [0, 1, 0, 0],
                 [0, 0, 0, 1],
                 [0, 0, 1, 0]], dtype=complex)

print("✅ Libraries loaded and quantum gates defined!")

✅ Libraries loaded and quantum gates defined!


## Part 1: Quantum Teleportation

### The Challenge
Alice wants to send an unknown quantum state |ψ⟩ to Bob. She can't just measure it (that would destroy the superposition) or clone it (no-cloning theorem). What can she do?

### The Solution: Quantum Teleportation Protocol

**Requirements:**
- 1 shared Bell pair (entangled qubits)
- 2 classical bits of communication
- Local quantum operations

**Key Insight**: We can transfer quantum information by cleverly using entanglement and classical communication!

In [ ]:
def create_bell_pair():
    """Create a Bell pair |Φ+⟩ = (|00⟩ + |11⟩)/√2"""
    # Start with |00⟩
    state = np.kron(ket0, ket0)
    # Apply H to first qubit
    state = np.kron(H, I) @ state
    # Apply CNOT
    state = CNOT @ state
    return state

def bell_measurement(state):
    """Perform Bell measurement on first two qubits of a 3-qubit state"""
    # Bell basis states
    bell_states = {
        '00': (np.kron(ket0, ket0) + np.kron(ket1, ket1)) / np.sqrt(2),  # |Φ+⟩
        '01': (np.kron(ket0, ket0) - np.kron(ket1, ket1)) / np.sqrt(2),  # |Φ-⟩
        '10': (np.kron(ket0, ket1) + np.kron(ket1, ket0)) / np.sqrt(2),  # |Ψ+⟩
        '11': (np.kron(ket0, ket1) - np.kron(ket1, ket0)) / np.sqrt(2),  # |Ψ-⟩
    }
    
    # Calculate probabilities for each Bell state
    probabilities = {}
    for outcome, bell_state in bell_states.items():
        # Project onto Bell state ⊗ I
        projector = np.kron(np.outer(bell_state, bell_state.conj()), I)
        prob = np.real(np.conj(state) @ projector @ state)
        probabilities[outcome] = prob
    
    return probabilities

# Demonstrate Bell pair creation
bell_pair = create_bell_pair()
print(r"Bell pair |Φ+⟩ created:")
print(f"State vector: {bell_pair}")
print(f"\nVerification: |00⟩ amplitude = {bell_pair[0]:.3f}")
print(f"             |11⟩ amplitude = {bell_pair[3]:.3f}")

In [ ]:
def visualize_teleportation_protocol():
    """Create an interactive visualization of the teleportation protocol"""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    # Left plot: Protocol diagram
    ax1.set_xlim(0, 10)
    ax1.set_ylim(0, 8)
    ax1.axis('off')
    ax1.set_title('Quantum Teleportation Protocol', fontsize=16, fontweight='bold')
    
    # Draw Alice's side
    alice_box = FancyBboxPatch((0.5, 4), 3, 3.5, 
                               boxstyle="round,pad=0.1", 
                               facecolor='lightblue', 
                               edgecolor='darkblue', 
                               linewidth=2)
    ax1.add_patch(alice_box)
    ax1.text(2, 6.8, 'Alice', fontsize=14, ha='center', fontweight='bold')
    
    # Draw Bob's side
    bob_box = FancyBboxPatch((6.5, 4), 3, 3.5, 
                             boxstyle="round,pad=0.1", 
                             facecolor='lightgreen', 
                             edgecolor='darkgreen', 
                             linewidth=2)
    ax1.add_patch(bob_box)
    ax1.text(8, 6.8, 'Bob', fontsize=14, ha='center', fontweight='bold')
    
    # Draw qubits
    ax1.add_patch(Circle((2, 6), 0.3, color='red'))
    ax1.text(2, 6, r'$|\psi\rangle$', fontsize=12, ha='center', va='center', color='white')
    
    ax1.add_patch(Circle((2, 5), 0.3, color='purple'))
    ax1.text(2, 5, 'A', fontsize=12, ha='center', va='center', color='white')
    
    ax1.add_patch(Circle((8, 5), 0.3, color='purple'))
    ax1.text(8, 5, 'B', fontsize=12, ha='center', va='center', color='white')
    
    # Draw entanglement
    ax1.plot([2.3, 7.7], [5, 5], 'purple', linewidth=3, linestyle='--', alpha=0.7)
    ax1.text(5, 5.3, 'Entangled', fontsize=10, ha='center')
    
    # Draw Bell measurement
    ax1.add_patch(Rectangle((1.5, 4.5), 1, 2, 
                           facecolor='yellow', 
                           edgecolor='orange', 
                           linewidth=2, 
                           alpha=0.7))
    ax1.text(2, 4.2, 'Bell\nMeas.', fontsize=10, ha='center', va='top')
    
    # Draw classical communication
    ax1.arrow(3.5, 4.5, 3, 0, head_width=0.2, head_length=0.2, 
             fc='gray', ec='gray', linewidth=2)
    ax1.text(5, 4.8, '2 classical bits', fontsize=10, ha='center')
    
    # Protocol steps
    ax1.text(5, 3, 'Protocol Steps:', fontsize=12, ha='center', fontweight='bold')
    steps = [
        r'1. Share Bell pair $|\Phi^+\rangle$ between Alice & Bob',
        r'2. Alice performs Bell measurement on $|\psi\rangle$ and her half',
        '3. Alice sends 2-bit measurement result to Bob',
        '4. Bob applies correction based on result'
    ]
    for i, step in enumerate(steps):
        ax1.text(5, 2.3 - i*0.4, step, fontsize=10, ha='center')
    
    # Right plot: State evolution
    ax2.axis('off')
    ax2.set_title('State Evolution', fontsize=16, fontweight='bold')
    
    # Create state evolution text
    state_text = [
        r"Initial: $|\psi\rangle \otimes |\Phi^+\rangle$",
        "After Bell measurement:",
        r"  00 → $|\psi\rangle$ (no correction)",
        r"  01 → $Z|\psi\rangle$ (apply Z)",
        r"  10 → $X|\psi\rangle$ (apply X)",
        r"  11 → $XZ|\psi\rangle$ (apply XZ)",
        "",
        r"Result: Bob has $|\psi\rangle$!"
    ]
    
    for i, text in enumerate(state_text):
        ax2.text(0.1, 0.9 - i*0.1, text, fontsize=12, 
                transform=ax2.transAxes, 
                fontweight='bold' if i == 0 or i == 7 else 'normal')
    
    plt.tight_layout()
    return fig

# Display the visualization
fig = visualize_teleportation_protocol()
plt.show()

### 🤔 Interactive Moment: What's Really Being Transmitted?

**Question for the audience**: In quantum teleportation, what exactly travels from Alice to Bob?

Think about it, then click below to reveal the answer!

In [ ]:
# Interactive reveal widget
button = widgets.Button(description="Reveal Answer")
output = widgets.Output()

def on_button_click(b):
    with output:
        output.clear_output()
        display(HTML("""
        <div style="background-color: #f0f8ff; padding: 20px; border-radius: 10px; border: 2px solid #4169e1;">
        <h3>Answer: Only Classical Information!</h3>
        <ul style="font-size: 14px;">
        <li><b>What travels:</b> 2 classical bits (the measurement outcome)</li>
        <li><b>What doesn't travel:</b> The qubit itself, or any physical particle</li>
        <li><b>Key insight:</b> The quantum information is "reconstructed" at Bob's location using:
            <ul>
            <li>Pre-shared entanglement</li>
            <li>Classical communication</li>
            <li>Local operations</li>
            </ul>
        </li>
        <li><b>No faster-than-light communication!</b> The classical bits must travel at or below light speed</li>
        </ul>
        </div>
        """))

button.on_click(on_button_click)
display(button, output)

In [ ]:
def quantum_teleportation(psi):
    """
    Simulate the complete quantum teleportation protocol
    
    Args:
        psi: The quantum state to teleport (2D array)
    
    Returns:
        measurement_outcome: The Bell measurement result
        bob_state: Bob's final state after correction
    """
    # Step 1: Create Bell pair between Alice and Bob
    bell_pair = create_bell_pair()
    
    # Step 2: Create full 3-qubit state |ψ⟩ ⊗ |Φ+⟩
    full_state = np.kron(psi, bell_pair)
    
    # Step 3: Apply CNOT between Alice's qubits (qubit 0 control, qubit 1 target)
    # We need to expand CNOT to act on 3 qubits
    CNOT_3q = np.kron(CNOT, I)
    full_state = CNOT_3q @ full_state
    
    # Step 4: Apply Hadamard to Alice's first qubit
    H_3q = np.kron(np.kron(H, I), I)
    full_state = H_3q @ full_state
    
    # Step 5: Measure Alice's qubits (simulate measurement)
    # Calculate probabilities for each outcome
    probs = bell_measurement(full_state)
    
    # For demonstration, we'll show all possible outcomes
    print("Bell measurement probabilities:")
    for outcome, prob in probs.items():
        print(f"  {outcome}: {prob:.3f}")
    
    # Simulate measurement (choose outcome based on probability)
    outcomes = list(probs.keys())
    probabilities = list(probs.values())
    measurement = np.random.choice(outcomes, p=probabilities)
    
    print(f"\nMeasurement outcome: {measurement}")
    
    # Step 6: Bob applies correction based on measurement
    corrections = {
        '00': I,      # No correction needed
        '01': Z,      # Apply Z gate
        '10': X,      # Apply X gate  
        '11': X @ Z   # Apply both X and Z
    }
    
    # Extract Bob's state (we would need to trace out Alice's qubits)
    # For simplicity, we'll apply the correction to the original state
    bob_state = corrections[measurement] @ psi
    
    return measurement, bob_state

# Test with a random state
theta = np.pi/3
test_state = np.cos(theta/2) * ket0 + np.sin(theta/2) * ket1
test_state = test_state / np.linalg.norm(test_state)

print(f"Original state |ψ⟩ = {test_state[0]:.3f}|0⟩ + {test_state[1]:.3f}|1⟩")
print("\nRunning teleportation protocol...\n")

measurement, final_state = quantum_teleportation(test_state)

print(f"\nBob's final state = {final_state[0]:.3f}|0⟩ + {final_state[1]:.3f}|1⟩")
print(f"\nFidelity with original: {abs(np.conj(test_state) @ final_state)**2:.6f}")

## 🔮 Part 2: Gate Teleportation (9 minutes)

### Motivation
What if instead of teleporting a state, we could teleport the *action* of a gate? This is the key to fault-tolerant quantum computing!

### The Protocol
1. Prepare a special "resource state" that encodes the gate
2. Use teleportation-like measurements
3. Apply corrections based on measurement outcomes

**Example**: Teleporting a T gate (important for universality)

In [ ]:
def visualize_gate_teleportation():
    """Visualize the gate teleportation protocol"""
    fig, ax = plt.subplots(figsize=(12, 8))
    ax.set_xlim(0, 12)
    ax.set_ylim(0, 10)
    ax.axis('off')
    ax.set_title('Gate Teleportation Protocol', fontsize=18, fontweight='bold')
    
    # Draw the circuit elements
    # Input state
    ax.add_patch(Circle((1, 7), 0.3, color='blue'))
    ax.text(1, 7, r'$|\psi\rangle$', fontsize=12, ha='center', va='center', color='white')
    
    # Resource state (magic state for T gate)
    ax.add_patch(Rectangle((3, 6), 2, 2, facecolor='gold', edgecolor='orange', linewidth=2))
    ax.text(4, 7, r'Magic$\newline$State$\newline$|T$\rangle$', fontsize=10, ha='center', va='center')
    
    # Entangling operation
    ax.plot([1.3, 3], [7, 7], 'k-', linewidth=2)
    ax.plot([5, 6], [7, 7], 'k-', linewidth=2)
    
    # Measurement
    ax.add_patch(Rectangle((6, 6.5), 1.5, 1, facecolor='lightcoral', edgecolor='darkred', linewidth=2))
    ax.text(6.75, 7, 'Meas', fontsize=10, ha='center', va='center')
    
    # Classical communication
    ax.arrow(7.5, 7, 2, 0, head_width=0.2, head_length=0.2, fc='gray', ec='gray')
    ax.text(8.5, 7.3, 'Classical', fontsize=9, ha='center')
    
    # Correction
    ax.add_patch(Rectangle((9.5, 6.5), 1.5, 1, facecolor='lightgreen', edgecolor='darkgreen', linewidth=2))
    ax.text(10.25, 7, 'Correct', fontsize=10, ha='center', va='center')
    
    # Output
    ax.add_patch(Circle((11.5, 7), 0.3, color='green'))
    ax.text(11.5, 7, r'$T|\psi\rangle$', fontsize=12, ha='center', va='center', color='white')
    
    # Key points
    ax.text(6, 5, 'Key Insights:', fontsize=14, ha='center', fontweight='bold')
    insights = [
        '• Magic states can be distilled to high fidelity',
        '• Only Clifford operations needed online',
        '• Foundation of fault-tolerant computing',
        '• Can teleport any gate with right resource'
    ]
    
    for i, insight in enumerate(insights):
        ax.text(6, 4.3 - i*0.5, insight, fontsize=11, ha='center')
    
    # Add circuit diagram
    ax.text(6, 1.5, 'Circuit Representation:', fontsize=12, ha='center', fontweight='bold')
    
    # Draw mini circuit
    y_circuit = 0.8
    ax.plot([2, 10], [y_circuit, y_circuit], 'k-', linewidth=1)
    ax.plot([2, 10], [y_circuit-0.3, y_circuit-0.3], 'k-', linewidth=1)
    
    # Gates in circuit
    ax.add_patch(Rectangle((3, y_circuit-0.15), 0.3, 0.3, facecolor='white', edgecolor='black'))
    ax.text(3.15, y_circuit, 'H', fontsize=8, ha='center', va='center')
    
    ax.plot([4, 4], [y_circuit-0.3, y_circuit], 'ko-', markersize=5)
    ax.add_patch(Circle((4, y_circuit), 0.1, facecolor='white', edgecolor='black'))
    
    ax.add_patch(Rectangle((5, y_circuit-0.4), 0.5, 0.5, facecolor='yellow', edgecolor='black'))
    ax.text(5.25, y_circuit-0.15, 'M', fontsize=8, ha='center', va='center')
    
    plt.tight_layout()
    return fig

fig = visualize_gate_teleportation()
plt.show()

In [ ]:
# T gate and magic state demonstration
T = np.array([[1, 0], [0, np.exp(1j*np.pi/4)]], dtype=complex)
magic_state = (ket0 + np.exp(1j*np.pi/4) * ket1) / np.sqrt(2)

print("T gate matrix:")
print(T)
print(f"\nMagic state |T⟩ = {magic_state[0]:.3f}|0⟩ + {magic_state[1]:.3f}|1⟩")
print(f"\nProperty: T|+⟩ = magic state")

# Verify the property
plus_state = (ket0 + ket1) / np.sqrt(2)
T_plus = T @ plus_state
print(f"T|+⟩ = {T_plus[0]:.3f}|0⟩ + {T_plus[1]:.3f}|1⟩")
print(f"Fidelity with magic state: {abs(np.conj(magic_state) @ T_plus)**2:.6f}")

## 📡 Part 3: Superdense Coding (8 minutes)

### The Amazing Claim
Send **2 classical bits** using only **1 qubit**! How? By exploiting pre-shared entanglement.

### The Protocol
1. Alice and Bob share a Bell pair
2. Alice encodes 2 bits by applying: I (00), X (01), Z (10), or XZ (11)
3. Alice sends her qubit to Bob
4. Bob performs Bell measurement to decode

This is the "reverse" of teleportation!

In [ ]:
def superdense_coding_demo():
    """Interactive demonstration of superdense coding"""
    
    def encode_message(bits):
        """Alice encodes 2 classical bits into her qubit"""
        # Start with Bell state |Φ+⟩
        bell_state = create_bell_pair()
        
        # Apply encoding based on bits
        if bits == '00':
            operation = np.kron(I, I)
            op_name = 'I'
        elif bits == '01':
            operation = np.kron(X, I)
            op_name = 'X'
        elif bits == '10':
            operation = np.kron(Z, I)
            op_name = 'Z'
        else:  # '11'
            operation = np.kron(X @ Z, I)
            op_name = 'XZ'
        
        encoded_state = operation @ bell_state
        return encoded_state, op_name
    
    def decode_message(state):
        """Bob decodes the message using Bell measurement"""
        # Apply CNOT
        state = CNOT @ state
        # Apply H ⊗ I
        state = np.kron(H, I) @ state
        
        # Measure in computational basis
        probs = np.abs(state)**2
        
        # Find which basis state has probability 1
        for i, prob in enumerate(probs):
            if prob > 0.99:
                return f"{i:02b}"
        return "Error"
    
    # Create interactive widget
    bit_selector = widgets.Dropdown(
        options=['00', '01', '10', '11'],
        description='Message:',
        value='00'
    )
    
    output = widgets.Output()
    
    def on_change(change):
        with output:
            output.clear_output()
            bits = change['new']
            
            print(f"Alice wants to send: {bits}")
            print("="*40)
            
            # Encode
            encoded_state, op_name = encode_message(bits)
            print(f"\n1. Alice applies {op_name} to her qubit")
            print(f"   Encoded state: {encoded_state}")
            
            # Decode
            decoded = decode_message(encoded_state.copy())
            print(f"\n2. Alice sends her qubit to Bob")
            print(f"\n3. Bob performs Bell measurement")
            print(f"   Decoded message: {decoded}")
            
            # Verify
            if decoded == bits:
                print(f"\n✅ Success! Bob received '{decoded}'")
            else:
                print(f"\n❌ Error in transmission!")
    
    bit_selector.observe(on_change, names='value')
    
    # Initial display
    on_change({'new': '00'})
    
    display(widgets.VBox([bit_selector, output]))

# Run the demonstration
print("🎮 Interactive Superdense Coding Demo")
print("Try different 2-bit messages and see how they're encoded/decoded!\n")
superdense_coding_demo()

In [ ]:
def compare_classical_vs_quantum():
    """Visual comparison of classical vs quantum channel capacity"""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))
    
    # Classical channel
    ax1.set_xlim(0, 10)
    ax1.set_ylim(0, 8)
    ax1.axis('off')
    ax1.set_title('Classical Channel', fontsize=16, fontweight='bold')
    
    # Alice
    ax1.add_patch(Rectangle((1, 3), 2, 2, facecolor='lightblue', edgecolor='darkblue', linewidth=2))
    ax1.text(2, 4, 'Alice', fontsize=12, ha='center', va='center')
    
    # Bob
    ax1.add_patch(Rectangle((7, 3), 2, 2, facecolor='lightgreen', edgecolor='darkgreen', linewidth=2))
    ax1.text(8, 4, 'Bob', fontsize=12, ha='center', va='center')
    
    # Channel
    ax1.arrow(3, 4, 4, 0, head_width=0.3, head_length=0.3, fc='gray', ec='gray', linewidth=3)
    ax1.text(5, 4.5, '1 bit', fontsize=14, ha='center', fontweight='bold')
    ax1.text(5, 3.5, 'Capacity: 1 bit', fontsize=12, ha='center')
    
    # Quantum channel with entanglement
    ax2.set_xlim(0, 10)
    ax2.set_ylim(0, 8)
    ax2.axis('off')
    ax2.set_title('Quantum Channel (Superdense Coding)', fontsize=16, fontweight='bold')
    
    # Alice
    ax2.add_patch(Rectangle((1, 3), 2, 2, facecolor='lightblue', edgecolor='darkblue', linewidth=2))
    ax2.text(2, 4, 'Alice', fontsize=12, ha='center', va='center')
    
    # Bob
    ax2.add_patch(Rectangle((7, 3), 2, 2, facecolor='lightgreen', edgecolor='darkgreen', linewidth=2))
    ax2.text(8, 4, 'Bob', fontsize=12, ha='center', va='center')
    
    # Entanglement
    ax2.plot([2, 8], [5.5, 5.5], 'purple', linewidth=3, linestyle='--')
    ax2.text(5, 5.8, 'Pre-shared entanglement', fontsize=10, ha='center', color='purple')
    
    # Quantum channel
    ax2.arrow(3, 4, 4, 0, head_width=0.3, head_length=0.3, fc='red', ec='red', linewidth=3)
    ax2.text(5, 4.5, '1 qubit', fontsize=14, ha='center', fontweight='bold')
    ax2.text(5, 3.5, 'Capacity: 2 bits!', fontsize=12, ha='center', color='red', fontweight='bold')
    
    # Key point
    ax2.text(5, 1.5, 'Key: Entanglement doubles capacity!', 
            fontsize=12, ha='center', fontweight='bold',
            bbox=dict(boxstyle='round,pad=0.5', facecolor='yellow', alpha=0.7))
    
    plt.tight_layout()
    return fig

fig = compare_classical_vs_quantum()
plt.show()

### 🤔 Audience Challenge

**What would happen if Alice and Bob had no pre-shared entanglement?**

Think about it, then run the cell below!

In [ ]:
# Challenge answer
print("Without entanglement:")
print("=" * 50)
print("❌ Superdense coding would NOT work!")
print("\nWhy?")
print("• A single qubit can only carry 1 bit of classical information")
print("• This is proven by Holevo's theorem")
print("• The 2-bit capacity comes from the 4 orthogonal Bell states")
print("• Without entanglement, we can't access this larger state space")
print("\nConclusion: Entanglement is the resource that enables superdense coding!")

## 🎯 Part 4: Deutsch-Jozsa Algorithm (15 minutes)

### The Problem
Given a black-box function f: {0,1}ⁿ → {0,1}, determine if it's:
- **Constant**: f(x) = 0 for all x, or f(x) = 1 for all x
- **Balanced**: f(x) = 0 for exactly half the inputs

**Classical**: Worst case needs 2ⁿ⁻¹ + 1 queries
**Quantum**: Only 1 query! 🤯

In [ ]:
def deutsch_jozsa_circuit(n_qubits):
    """
    Visualize the Deutsch-Jozsa circuit for n qubits
    """
    fig, ax = plt.subplots(figsize=(14, 8))
    ax.set_xlim(0, 14)
    ax.set_ylim(-1, n_qubits + 2)
    ax.axis('off')
    ax.set_title(f'Deutsch-Jozsa Circuit (n={n_qubits})', fontsize=18, fontweight='bold')
    
    # Draw qubit lines
    for i in range(n_qubits + 1):
        y = n_qubits - i
        ax.plot([1, 13], [y, y], 'k-', linewidth=1)
        if i < n_qubits:
            ax.text(0.5, y, r'$|0\rangle$', fontsize=12, ha='center', va='center')
        else:
            ax.text(0.5, y, r'$|1\rangle$', fontsize=12, ha='center', va='center')
    
    # Initial Hadamards
    for i in range(n_qubits + 1):
        y = n_qubits - i
        ax.add_patch(Rectangle((2, y-0.2), 0.6, 0.4, 
                              facecolor='lightblue', 
                              edgecolor='darkblue', 
                              linewidth=2))
        ax.text(2.3, y, 'H', fontsize=12, ha='center', va='center')
    
    # Oracle
    ax.add_patch(Rectangle((5, -0.5), 2, n_qubits + 1.5, 
                          facecolor='orange', 
                          edgecolor='darkorange', 
                          linewidth=2, 
                          alpha=0.7))
    ax.text(6, n_qubits/2, 'Oracle\nUf', fontsize=14, ha='center', va='center', fontweight='bold')
    
    # Final Hadamards (only on input qubits)
    for i in range(n_qubits):
        y = n_qubits - i
        ax.add_patch(Rectangle((9, y-0.2), 0.6, 0.4, 
                              facecolor='lightblue', 
                              edgecolor='darkblue', 
                              linewidth=2))
        ax.text(9.3, y, 'H', fontsize=12, ha='center', va='center')
    
    # Measurements
    for i in range(n_qubits):
        y = n_qubits - i
        ax.add_patch(Rectangle((11, y-0.25), 0.8, 0.5, 
                              facecolor='lightcoral', 
                              edgecolor='darkred', 
                              linewidth=2))
        ax.text(11.4, y, 'M', fontsize=12, ha='center', va='center')
        ax.arrow(11.8, y, 0.8, 0, head_width=0.1, head_length=0.1, fc='black', ec='black')
        ax.text(12.8, y, f'c{i}', fontsize=10, ha='center', va='center')
    
    # Label sections
    ax.text(2.3, -0.8, 'Create\nsuperposition', fontsize=10, ha='center', color='darkblue')
    ax.text(6, -0.8, 'Query oracle\n(phase kickback)', fontsize=10, ha='center', color='darkorange')
    ax.text(9.3, -0.8, 'Interference', fontsize=10, ha='center', color='darkblue')
    ax.text(11.4, -0.8, 'Measure', fontsize=10, ha='center', color='darkred')
    
    plt.tight_layout()
    return fig

# Show circuit for n=3
fig = deutsch_jozsa_circuit(3)
plt.show()

In [ ]:
def create_oracle(f_type, n):
    """
    Create an oracle for the Deutsch-Jozsa algorithm
    
    Args:
        f_type: 'constant_0', 'constant_1', or 'balanced'
        n: number of input qubits
    """
    size = 2**(n+1)
    oracle = np.eye(size, dtype=complex)
    
    if f_type == 'constant_0':
        # f(x) = 0 for all x, so oracle does nothing
        pass
    elif f_type == 'constant_1':
        # f(x) = 1 for all x, so flip the ancilla for all inputs
        for i in range(size//2):
            oracle[2*i, 2*i] = 0
            oracle[2*i, 2*i+1] = 1
            oracle[2*i+1, 2*i] = 1
            oracle[2*i+1, 2*i+1] = 0
    elif f_type == 'balanced':
        # f(x) = 1 for half the inputs
        # Simple example: f(x) = 1 if first bit is 1
        for i in range(size//2):
            if i >= size//4:
                oracle[2*i, 2*i] = 0
                oracle[2*i, 2*i+1] = 1
                oracle[2*i+1, 2*i] = 1
                oracle[2*i+1, 2*i+1] = 0
    
    return oracle

def deutsch_jozsa(oracle, n):
    """
    Run the Deutsch-Jozsa algorithm
    
    Returns: True if constant, False if balanced
    """
    # Initialize state |0...0⟩|1⟩
    state = np.zeros(2**(n+1), dtype=complex)
    state[1] = 1  # |00...01⟩
    
    # Apply Hadamard to all qubits
    for i in range(n+1):
        H_n = np.eye(2**(n+1), dtype=complex)
        for j in range(2**(n+1)):
            bit_i = (j >> i) & 1
            bit_flip = j ^ (1 << i)
            if bit_i == 0:
                H_n[j, j] = 1/np.sqrt(2)
                H_n[j, bit_flip] = 1/np.sqrt(2)
            else:
                H_n[j, bit_flip] = 1/np.sqrt(2)
                H_n[j, j] = -1/np.sqrt(2)
        state = H_n @ state
    
    # Apply oracle
    state = oracle @ state
    
    # Apply Hadamard to input qubits (not ancilla)
    for i in range(1, n+1):
        H_n = np.eye(2**(n+1), dtype=complex)
        for j in range(2**(n+1)):
            bit_i = (j >> i) & 1
            bit_flip = j ^ (1 << i)
            if bit_i == 0:
                H_n[j, j] = 1/np.sqrt(2)
                H_n[j, bit_flip] = 1/np.sqrt(2)
            else:
                H_n[j, bit_flip] = 1/np.sqrt(2)
                H_n[j, j] = -1/np.sqrt(2)
        state = H_n @ state
    
    # Measure input qubits
    prob_all_zeros = abs(state[0])**2 + abs(state[1])**2
    
    return prob_all_zeros > 0.99

# Test the algorithm
n = 3
print(f"Testing Deutsch-Jozsa algorithm with n={n} qubits\n")

for f_type in ['constant_0', 'constant_1', 'balanced']:
    oracle = create_oracle(f_type, n)
    result = deutsch_jozsa(oracle, n)
    
    print(f"Oracle type: {f_type}")
    print(f"Algorithm says: {'CONSTANT' if result else 'BALANCED'}")
    print(f"Correct? {'✅' if (result and 'constant' in f_type) or (not result and f_type == 'balanced') else '❌'}")
    print()

### 📊 Visual Demonstration: Interference Pattern

In [ ]:
def visualize_interference():
    """Visualize how interference leads to the measurement outcome"""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))
    
    # Constant function visualization
    ax1.set_title('Constant Function: Constructive Interference', fontsize=14, fontweight='bold')
    ax1.set_xlim(-0.5, 3.5)
    ax1.set_ylim(-0.5, 1.5)
    ax1.set_xlabel('Computational Basis States', fontsize=12)
    ax1.set_ylabel('Amplitude', fontsize=12)
    
    # Amplitudes for constant function (all amplitudes add up at |00...0⟩)
    states = [r'$|00\rangle$', r'$|01\rangle$', r'$|10\rangle$', r'$|11\rangle$']
    amplitudes_const = [1, 0, 0, 0]
    
    bars1 = ax1.bar(states, amplitudes_const, color='green', alpha=0.7, edgecolor='darkgreen', linewidth=2)
    ax1.axhline(y=0, color='black', linewidth=1)
    ax1.text(0, 1.1, r'All paths interfere$\newline$constructively at $|00\rangle$', 
            fontsize=11, ha='center', bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.7))
    
    # Balanced function visualization
    ax2.set_title('Balanced Function: Destructive Interference', fontsize=14, fontweight='bold')
    ax2.set_xlim(-0.5, 3.5)
    ax2.set_ylim(-0.5, 1.5)
    ax2.set_xlabel('Computational Basis States', fontsize=12)
    ax2.set_ylabel('Amplitude', fontsize=12)
    
    # Amplitudes for balanced function (destructive interference at |00...0⟩)
    amplitudes_balanced = [0, 0.5, 0.5, 0.5]
    
    bars2 = ax2.bar(states, amplitudes_balanced, color='red', alpha=0.7, edgecolor='darkred', linewidth=2)
    ax2.axhline(y=0, color='black', linewidth=1)
    ax2.text(2, 0.7, r'Paths cancel out$\newline$at $|00\rangle$', 
            fontsize=11, ha='center', bbox=dict(boxstyle='round', facecolor='lightcoral', alpha=0.7))
    
    plt.tight_layout()
    return fig

fig = visualize_interference()
plt.show()

### 🎯 Mini Quiz: Understanding the Algorithm

In [ ]:
# Interactive quiz
question = widgets.RadioButtons(
    options=['The function is constant', 
             'The function is balanced', 
             'The measurement failed', 
             'We need to measure again'],
    description='Q: If we measure all zeros in Deutsch-Jozsa, what does it mean?',
    style={'description_width': 'initial'},
    layout={'width': 'max-content'}
)

button = widgets.Button(description="Check Answer")
output = widgets.Output()

def check_answer(b):
    with output:
        output.clear_output()
        if question.value == 'The function is constant':
            print("✅ Correct!")
            print("\nExplanation:")
            print("• All zeros means all amplitude concentrated at |00...0⟩")
            print("• This happens through constructive interference")
            print("• Only possible if f(x) is the same for all x (constant)")
        else:
            print("❌ Not quite. Try again!")
            print("\nHint: Think about interference patterns...")

button.on_click(check_answer)
display(question, button, output)

## 🌐 Big Picture and Connections (8 minutes)

### How These Protocols Connect

1. **Teleportation ↔ Superdense Coding**: These are *dual* protocols!
   - Teleportation: 1 qubit + 2 classical bits → 1 qubit of quantum info
   - Superdense: 1 qubit + entanglement → 2 classical bits

2. **Gate Teleportation → Fault-Tolerant Computing**
   - Foundation for implementing non-Clifford gates fault-tolerantly
   - Magic state distillation is key to universal quantum computing

3. **Deutsch-Jozsa → Quantum Algorithms**
   - Same principles lead to Grover's search and Simon's algorithm
   - Shows power of quantum parallelism and interference

In [ ]:
def create_connections_diagram():
    """Create a visual map of protocol connections"""
    fig, ax = plt.subplots(figsize=(12, 8))
    ax.set_xlim(0, 10)
    ax.set_ylim(0, 10)
    ax.axis('off')
    ax.set_title('Quantum Protocol Connections', fontsize=18, fontweight='bold')
    
    # Protocol boxes
    protocols = [
        {'name': 'Teleportation', 'pos': (2, 7), 'color': 'lightblue'},
        {'name': 'Superdense\nCoding', 'pos': (8, 7), 'color': 'lightgreen'},
        {'name': 'Gate\nTeleportation', 'pos': (2, 4), 'color': 'lightyellow'},
        {'name': 'Deutsch-Jozsa', 'pos': (8, 4), 'color': 'lightcoral'},
        {'name': 'Fault-Tolerant\nComputing', 'pos': (2, 1), 'color': 'lavender'},
        {'name': 'Grover/Simon', 'pos': (8, 1), 'color': 'peachpuff'}
    ]
    
    for p in protocols:
        box = FancyBboxPatch((p['pos'][0]-1, p['pos'][1]-0.5), 2, 1, 
                            boxstyle="round,pad=0.1", 
                            facecolor=p['color'], 
                            edgecolor='black', 
                            linewidth=2)
        ax.add_patch(box)
        ax.text(p['pos'][0], p['pos'][1], p['name'], 
               fontsize=11, ha='center', va='center', fontweight='bold')
    
    # Connections
    # Teleportation <-> Superdense coding
    ax.annotate('', xy=(7, 7), xytext=(3, 7),
                arrowprops=dict(arrowstyle='<->', lw=2, color='purple'))
    ax.text(5, 7.3, 'Dual protocols', fontsize=10, ha='center', color='purple')
    
    # Gate teleportation -> Fault tolerance
    ax.annotate('', xy=(2, 2), xytext=(2, 3.5),
                arrowprops=dict(arrowstyle='->', lw=2, color='darkblue'))
    ax.text(2.5, 2.75, 'Enables', fontsize=10, ha='left', color='darkblue')
    
    # Deutsch-Jozsa -> Grover/Simon
    ax.annotate('', xy=(8, 2), xytext=(8, 3.5),
                arrowprops=dict(arrowstyle='->', lw=2, color='darkred'))
    ax.text(8.5, 2.75, 'Inspires', fontsize=10, ha='left', color='darkred')
    
    # Key resources
    ax.text(5, 9, 'Key Resources:', fontsize=14, ha='center', fontweight='bold')
    ax.text(5, 8.5, '• Entanglement', fontsize=12, ha='center')
    ax.text(5, 8.1, '• Interference', fontsize=12, ha='center')
    ax.text(5, 7.7, '• Measurement', fontsize=12, ha='center')
    
    plt.tight_layout()
    return fig

fig = create_connections_diagram()
plt.show()

## 🎬 Wrap-Up: The Power of Quantum Weirdness

### Final Question
**Which of these protocols would still work without entanglement? Which rely on interference?**

Take a moment to think, then explore the answer below!

In [ ]:
# Final summary table
data = {
    'Protocol': ['Teleportation', 'Gate Teleportation', 'Superdense Coding', 'Deutsch-Jozsa'],
    'Needs Entanglement?': ['✅ Yes', '✅ Yes', '✅ Yes', '❌ No'],
    'Uses Interference?': ['❌ No', '❌ No', '❌ No', '✅ Yes'],
    'Key Resource': ['Bell pairs', 'Magic states', 'Bell pairs', 'Superposition']
}

# Create and style the table
import pandas as pd
df = pd.DataFrame(data)

# Display with custom styling
def style_table(val):
    if '✅' in str(val):
        return 'background-color: #90EE90'
    elif '❌' in str(val):
        return 'background-color: #FFB6C1'
    return ''

styled_df = df.style.applymap(style_table).set_properties(**{
    'text-align': 'center',
    'font-size': '14px',
    'border': '2px solid black'
}).set_table_styles([{
    'selector': 'th',
    'props': [('font-size', '16px'), ('text-align', 'center'), ('font-weight', 'bold')]
}])

display(HTML("<h3>Protocol Resource Requirements</h3>"))
display(styled_df)

## 📚 Resources for Further Learning

### Recommended Resources:
1. **IBM Qiskit Textbook**: Interactive tutorials on all these protocols
2. **Nielsen & Chuang**: Chapter 1 (Teleportation, Superdense Coding), Chapter 3 (Deutsch-Jozsa)
3. **Quantum Computing: An Applied Approach** by Hidary: Great visual explanations
4. **Microsoft Quantum Development Kit**: Hands-on implementations

### 🔧 Optional Homework
1. **Implement Deutsch-Jozsa** for n=2 in your favorite quantum framework
2. **Compare classical vs quantum**: Write classical superdense coding and see why it's limited to 1 bit
3. **Trace through teleportation**: Work out the math for each measurement outcome

### Key Takeaways
- 🌟 Entanglement enables impossible classical tasks
- 🌊 Interference creates exponential speedups
- 📏 Measurement is both limitation and resource
- 🔗 These simple protocols are building blocks for complex algorithms

Thank you for exploring the quantum world with me! 🚀

**Remember**: *"Quantum mechanics isn't just weird — it's useful!"*